# Compare tweets and party programs

https://github.com/adsieg/text_similarity/blob/master/Different%20Embeddings%20%2B%20Cosine%20Similarity%20%2B%20HeatMap%20illustration.ipynb 

https://www.machinelearningplus.com/nlp/cosine-similarity/


In [73]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime 
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import gensim
from gensim.models import Word2Vec
import re
from nltk.corpus import stopwords

In [20]:
programme = open('party_programmes/gruene.txt', "r").read()

programme = programme.strip()

In [74]:
#Cleaning text 

#also stole this from: https://github.com/adsieg/text_similarity/blob/master/Different%20Embeddings%20%2B%20Cosine%20Similarity%20%2B%20HeatMap%20illustration.ipynb
def preprocess(raw_text):

    # keep only words
    letters_only_text = re.sub("[^a-zA-Z]", " ", raw_text)

    # convert to lower case and split 
    words = letters_only_text.lower().split()

    # remove stopwords
    stopword_set = set(stopwords.words("german"))
    cleaned_words = list(set([w for w in words if w not in stopword_set]))

    return cleaned_words

gruene_cleaned = preprocess(programme)

In [83]:
for_comp = [gruene_cleaned, "Karl Lauterbach soll Gesundheitsminister werden.", "Klimawandel ist ein wichtiges Thema"] #comparig the program with two sample sentences

In [84]:
# Create the Document Term Matrix
#stole this code from here: https://www.machinelearningplus.com/nlp/cosine-similarity/  


count_vectorizer = CountVectorizer() ## need to remove stopwords and use TfidfVectorizer instead?? 
sparse_matrix = count_vectorizer.fit_transform(for_comp)

# OPTIONAL: Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(doc_term_matrix, 
                  columns=count_vectorizer.get_feature_names(), 
                  index=['gruene', 'Lauterbach', 'Klima'])
df


AttributeError: 'list' object has no attribute 'lower'

In [59]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(df, df))

[[1.         0.06450307 0.06484987]
 [0.06450307 1.         0.        ]
 [0.06484987 0.         1.        ]]


In [39]:
# get pre-trained model

# downloaded a German word embedding model from http://devmount.github.io/GermanWordEmbeddings/#download 
# das github von dem dude: https://github.com/devmount/GermanWordEmbeddings/blob/master/code/pca.ipynb

model = gensim.models.KeyedVectors.load_word2vec_format("/Users/johannes/Downloads/german.model", binary=True) # i know you guys didn't download it yet but just testing things. 

What I would need to do, to make this work:
- Get a better embedding model for German language (given that this model doesn't even contain Baerbock)
- Loop each word into the model and calculate the mean (or weighted mean)
- then calculate the scipy distance 
- this can work. currently the main thing is the German Language Model.

In [102]:
cosine_similarity(np.asarray(model["Gruene"]).reshape(1,-1), np.asarray(model["Osten"]).reshape(1,-1))

array([[0.26798216]], dtype=float32)

# Trying with the German FastText Wiki Model

In [106]:
import fasttext

In [107]:
ft_model = fasttext.load_model("/Users/johannes/Downloads/wiki.de/wiki.de.bin")

In [112]:
# Simple test how a vector looks. it is pretty

ft_model['Gruene']


array([ 3.10224563e-01,  9.07221064e-02,  2.96998650e-01,  9.94986519e-02,
        4.12711173e-01,  3.06719720e-01,  3.04239661e-01, -1.50212556e-01,
       -2.94918567e-03,  1.44883946e-01, -1.74862891e-01,  4.39091325e-02,
        1.02864705e-01, -1.86042026e-01,  6.81281239e-02,  3.77346456e-01,
        3.42065930e-01, -1.65252537e-01,  1.44632027e-01, -9.03984457e-02,
        5.99124171e-02,  6.94528371e-02,  3.01870942e-01, -1.99355662e-01,
        1.44362422e-02,  8.07003602e-02, -1.27326056e-01, -4.43713665e-02,
       -2.21778452e-01, -1.40716955e-01,  2.43051007e-01,  3.54338884e-01,
       -2.64221728e-01, -3.55228096e-01, -8.11479520e-03, -6.51899278e-01,
        1.09513797e-01, -2.64682267e-02, -9.74089876e-02, -8.75184536e-02,
        1.39009720e-02,  2.01167420e-01,  2.32282698e-01,  1.12420492e-01,
        2.31429804e-02,  4.74265903e-01,  3.52653295e-01,  4.32717443e-01,
        4.74041477e-02, -4.01689470e-01,  5.54402649e-01, -1.78509220e-01,
       -1.61929339e-01, -

In [120]:
cleaned_programm = gensim.utils.simple_preprocess(programme)
cleaned_tweet = gensim.utils.simple_preprocess("Karl Lauterbach hat ne geile Fliege und die ist rot")

In [123]:
#anscheinend hat dieses kack model keine ahnung wer Baerbock ist...
program_vector = np.mean([ft_model[word] for word in cleaned_programm], axis = 0).reshape(1,-1)
tweet_vector = np.mean([ft_model[word] for word in cleaned_tweet], axis = 0).reshape(1,-1)

In [124]:
cosine_similarity(program_vector, tweet_vector)

array([[0.71076447]], dtype=float32)